In [ ]:
import pandas as pd
from os.path import join

from sklearn import svm
from pandas import DataFrame
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
import os
os.chdir('/home/potusvn/Projects/kickstarter_prediction')

## Check the with SVM Quantile

In [ ]:
train_quantile = pd.read_csv('data/data_quantile_transform/train_quantile_transform.csv', encoding='latin1', low_memory=True)
test_quantile = pd.read_csv('data/data_quantile_transform/test_quantile_transform.csv', encoding='latin1', low_memory=True)
val_quantile = pd.read_csv('data/data_quantile_transform/val_quantile_transform.csv', encoding='latin1', low_memory=True)

In [ ]:
train_quantile_large = pd.concat([train_quantile,val_quantile], axis=0)

In [ ]:
train_x = train_quantile_large.drop(['success'], axis=1)
train_y = train_quantile_large.success
test_x = test_quantile.drop(['success'], axis=1)
test_y = test_quantile.success

In [ ]:
train_x.shape

In [ ]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
supp_vec_machine = svm.SVC()
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('svm', supp_vec_machine)])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
#     'pca__n_components': [int(i*221/100) for i in range(20, 100, 20)],
    'svm__kernel' : ['rbf', 'poly', 'linear'],
#     'svm__C' : [0.001, 0.01, 0.1, 1, 10],
#     'svm__gamma': [0.001, 0.01, 0.1, 1]
    
}
search = GridSearchCV(pipe, param_grid, iid=False, cv=5,
                      return_train_score=False, n_jobs=-1)
search.fit(train_x, train_y)

In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
search

In [ ]:
# Plot the PCA spectrum
pca.fit(X_digits)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(pca.explained_variance_ratio_, linewidth=2)
ax0.set_ylabel('PCA explained variance')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,